In [1]:
import requests
import json
import datetime
import time
from dateutil import parser

In [17]:
client_id = "izafpjfe3fm11oqqido7mf8qzbyiw1"
headers = {'Accept': 'application/vnd.twitchtv.v5+json',
          'Client-ID': client_id}
url = 'https://api.twitch.tv/kraken/games/top?limit=50'

In [3]:
def get_top_games(url, header):
    now = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    game_list = [] #creo una lista vuota
    game_list.append({'timestamp':now})
    res = requests.get(url, headers = headers) #eseguo una richiesta all'API di Twitch
    game_list.extend(res.json()['top'])
    return(game_list)

In [4]:
def lighten_data(game_list):
    for item in game_list:
        try:
            del item['game']['locale']
            del item['game']['localized_name']
            del item['game']['box']['small']
            del item['game']['box']['medium']
            del item['game']['box']['template']
            del item['game']['box']['large']
            del item['game']['logo']['small']
            del item['game']['logo']['medium']
            del item['game']['logo']['template']
            del item['game']['logo']['large']
            del item['game']['logo']
            del item['game']['box']
        except KeyError:
                pass
    return(game_list)

In [5]:
def del_not_games(game_list): 
    d = {} #dizionario vuoto
    d['game'] = game_list[1:50]
    new_data = []
    new_data.append(game_list[0])
    for item in d['game']:
        if item['game']['giantbomb_id'] != 0:
            new_data.append(item)
    return(new_data) 

In [6]:
def add_ranking(game_list):
    n = len(game_list)
    ranking = 1
    for item in game_list[1:n]:
        item['game']['ranking'] = ranking
        ranking = ranking + 1
    return(game_list)

In [7]:
def add_partday(game_list):
    timestamp = game_list[0]['timestamp']
    parsed_timestamp = parser.parse(timestamp)
    actual_time = parsed_timestamp.strftime("%H:%M:%S")
    ore_mattino = ['06','07','08','09','10','11']
    ore_pomeriggio = ['12', '13','14', '15', '16', '17']
    ore_sera = ['18', '19', '20', '21', '22', '23']
    ore_notte = ['00', '01', '01', '02', '03', '04', '05']
    if actual_time[0:2] in ore_mattino:
        game_list.append({'part_day':'morning'})
    if actual_time[0:2] in ore_pomeriggio:
        game_list.append({'part_day':'afternoon'})
    if actual_time[0:2] in ore_sera:
        game_list.append({'part_day':'evening'})
    if actual_time[0:2] in ore_notte:
        game_list.append({'part_day':'night'})
    return(game_list)

In [8]:
def reashape_twitch(game_list):
    n = len(game_list)
    now = game_list[0]['timestamp']
    part_day = game_list[n-1]['part_day']
    for item in game_list[1:n-1]:
        item['timestamp'] = now
        item['part_day'] = part_day
    del game_list[0]
    del game_list[len(game_list)-1]
    return(game_list)

In [18]:
lista = []
while True:
    game_list = get_top_games(url, headers)
    game_list = lighten_data(game_list)
    game_list = del_not_games(game_list)
    game_list = add_ranking(game_list)
    game_list = add_partday(game_list)
    game_list = reashape_twitch(game_list)
    for item in game_list:
        message = {}
        message['timestamp'] = item['timestamp']
        message['part_of_day'] = item['part_day']
        message['name'] = item['game']['name']
        message['giantbomb_id'] = item['game']['giantbomb_id']
        message['viewers'] = item['viewers']
        message['channels'] = item['channels']
        message['ranking'] = item['game']['ranking']
        print(message)
        lista.append(message)
    time.sleep(60)

{'timestamp': '2020-05-04 15:34:02', 'part_of_day': 'afternoon', 'name': 'VALORANT', 'giantbomb_id': 77445, 'viewers': 307850, 'channels': 6326, 'ranking': 1}
{'timestamp': '2020-05-04 15:34:02', 'part_of_day': 'afternoon', 'name': 'League of Legends', 'giantbomb_id': 24024, 'viewers': 210887, 'channels': 4654, 'ranking': 2}
{'timestamp': '2020-05-04 15:34:02', 'part_of_day': 'afternoon', 'name': 'Fortnite', 'giantbomb_id': 37030, 'viewers': 139772, 'channels': 11880, 'ranking': 3}
{'timestamp': '2020-05-04 15:34:02', 'part_of_day': 'afternoon', 'name': 'Dota 2', 'giantbomb_id': 32887, 'viewers': 109963, 'channels': 1229, 'ranking': 4}
{'timestamp': '2020-05-04 15:34:02', 'part_of_day': 'afternoon', 'name': 'Counter-Strike: Global Offensive', 'giantbomb_id': 36113, 'viewers': 107631, 'channels': 2614, 'ranking': 5}
{'timestamp': '2020-05-04 15:34:02', 'part_of_day': 'afternoon', 'name': 'Grand Theft Auto V', 'giantbomb_id': 36765, 'viewers': 84177, 'channels': 2136, 'ranking': 6}
{'tim

KeyboardInterrupt: 

In [19]:
with open(r'C:\Users\39331\Desktop\prova1.json', 'w') as outfile:
    json.dump(lista, outfile)